In [139]:
# 导入需要的包
import random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import base64
import  requests
import re
import pandas as pd
import time

In [140]:
HTTP_HEADER ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}


In [141]:
# 加密专用
class Encrypter:

    def __init__(self):
        # 加密思路：
        # 1、使用ase_iv,ase_mode,ase_key,对需要提交的字符串进行第一次加密;
        # 2、随机生成一个16位的秘钥，结合aes算法，对数据进行第二次加密;  加密后得到 => encText
        # 3、将随机生成的秘钥，采用非对称加密的方式，使用公钥加密；加密后得到 => encKey
        # 4、将2、3得到的数据作为form表单的参数提交至服务器
        self.sequence = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
        self.aes_iv = '0102030405060708'
        self.aes_mode = AES.MODE_CBC
        self.aes_key = '0CoJUm6Qyw8W8jud'
        self.rsa_public_key = '010001'
        self.rsa_modulus = '00e0b509f6259df8642dbc35662901'\
                    '477df22677ec152b5ff68ace615bb7b7'\
                    '25152b3ab17a876aea8a5aa76d2e417629'\
                    'ec4ee341f56135fccf695280104e0312e'\
                    'cbda92557c93870114af6c9d05c4f7f0c'\
                    '3685b7a46bee255932575cce10b424d813c'\
                    'fe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'

    def random_ase_key(self):
        random_key = random.sample(self.sequence,16)
        random_key = ''.join(random_key)
        return random_key
    
    def aes_encrypt(self,plain_text,mode=AES.MODE_CBC,iv=None,key=None):
        # data padding
        # data_to_padd : need a byte objectencode return a bytes object,
        # The block boundary to use for padding
        plain_text = pad(data_to_pad=plain_text.encode(),block_size= AES.block_size)
        # create a encrypt object
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        # get the encrypted data
        encrypted_data = aes.encrypt(plaintext=plain_text)
        # use base64 to transfer the encrypted data
        # why ? 参考：
        result = base64.b64encode(encrypted_data).decode()
        return result

    def rsa_encrypt(self,data, public_key, modulus):
        # 部分情况下的运算会导致出现位数不足256位，需要进行填充
        num = pow(int(data[::-1].encode().hex(), 16), int(public_key, 16), int(modulus, 16))
        result = format(num, 'x').zfill(256)
        return result
    def aes_decrypt(self,encode_text,mode=AES.MODE_CBC,iv=None,key=None):
        # use base64 decode the data
        encode_data = base64.b64decode(encode_text.encode())
        # 
        aes = AES.new(key=key.encode(),mode=mode,iv=iv.encode())
        decrypted_data = aes.decrypt(encode_data)
        decrypted_data = unpad(decrypted_data,AES.block_size)
        return decrypted_data.decode()
    
    def get_encrypt(self,plain_text):
        random_key = self.random_ase_key()
        encode_data = self.aes_encrypt(plain_text=plain_text,mode=self.aes_mode,iv=self.aes_iv,key=self.aes_key)
        # 加密后再使用random_key加密
        encText = self.aes_encrypt(plain_text=encode_data,mode=self.aes_mode,iv=self.aes_iv,key=random_key)
        # 使用公钥对 random_key 进行加密，加密后传给服务器
        encKey = self.rsa_encrypt(random_key,self.rsa_public_key,self.rsa_modulus)

        form_data = {
            'encSecKey':encKey, #为何有时候是255位？计算问题...自动填充0即可
            'params':encText
            }
        return form_data

In [142]:
t = Encrypter()
form_data = t.get_encrypt('{"s":"张国荣","limit":"8","csrf_token":""}')

In [143]:
print(type('ss'.encode()))
# print('你'.encode())
# len('你好'.encode()) # encode返回byte形式,默认是utf-8
# print('你好'.encode(encoding='GBK'))
# len('你好'.encode(encoding='GBK'))
t = '你好'.encode(encoding='GBK')
print(t)
# print(t.decode(encoding='utf-8')) error
print(t.decode(encoding='GBK'))

<class 'bytes'>
b'\xc4\xe3\xba\xc3'
你好


In [144]:
format(11111,'x')

'2b67'

In [145]:
print(form_data)

{'encSecKey': '4f76fe0b4fbfbd8e8d64ab592b799f2355eaa3bcee7433da25eaab9f6432171185d5ca9ee3f3d646444be460843d030f421cb35e73a2d5411a74f88897b67a346e20fcf831b02f384cc1039e7843a85ef86ec4a6940d0d1f22c247018ec25b791de68327060882d8ef3f066c2ad1a2195cbb598c1ae35b35b1b567f8a7e349c4', 'params': 'gnlx9dtPYBfF5x1B8g15lTGk64WU5ACm78+hHb13oHzSNgs89Kvrg2rRK9OorA0IJFp5Oz3IJJp1Viu6pemA+OEBQkczeBm324QByk53gEw='}


In [146]:
HTTP_HEADER ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                                      'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

def get_text_post(url,data):
    html = requests.post(url=url,data = data,headers=HTTP_HEADER)

    return html.text

# text = get_text_post('https://music.163.com/weapi/search/suggest/web?csrf_token=',form_data)
# text

In [147]:
# import re
# text = re.sub(r'null|false','None',text)
# text = eval(text)

In [148]:
# text['result'].keys()

In [149]:
# for item in text['result']['artists']:
#     print(item['id'])
#     print(item['name'])

In [150]:
# for item in text['result']['songs']:
#     # print(item)
#     print(item['id'])
#     print(item['name'])

In [151]:
# def handle_albums(albums):
#     result = {}
#     for i in albums:
#         result[i['id']] = result[i['name']]
#     return result

In [152]:
# for i in  text['result']['albums']:
#     print(i['id'])
#     print(i['name'])

In [153]:
class MusicDownloader:
    def __init__(self) :
        self.url = 'https://music.163.com/weapi/cloudsearch/get/web?csrf_token='
        self.download_url = 'https://music.163.com/weapi/song/enhance/player/url/v1?csrf_token='
        self.comment_url = 'https://music.163.com/weapi/comment/resource/comments/get?csrf_token='

    def get_dic(self,search_data):
        text = get_text_post(self.url,search_data)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        result_dic = {}
        #print(text)
        #print(text.keys())
        for song in text['result']['songs']:
            result_dic[song['id']] = song['name']
        return result_dic
    
    def download_music(self,music,music_name):
        text = get_text_post(self.download_url,music)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        url = text['data'][0]['url']

        data = requests.get(url)
        with open('download/'+music_name+'.m4a','wb') as f:
            f.write(data.content)

        return text
        
    def download_hot_comment(self,music,music_name):
        text = get_text_post(self.comment_url,music)
        text = re.sub(r'null|false|true','None',text)
        text = eval(text)
        music_hot_comments = []
        if text['data']['hotComments'] == None:
            return None
        for comment in text['data']['hotComments']:
            user = comment['user']['nickname']
            content =  comment['content']
            likedCount = comment['likedCount']
            music_hot_comments.append([user,content,likedCount])
        user = []
        content = []
        likedCount = []
        for i in range(len(music_hot_comments)):
            user.append(music_hot_comments[i][0].encode('utf-8','replace').decode('utf-8'))
            content.append(music_hot_comments[i][1].encode('utf-8','replace').decode('utf-8'))
            likedCount.append(music_hot_comments[i][2])
        data = pd.DataFrame(
            { 'user':user,
            'content':content,
            'likedCount':likedCount}
        )
        print(data)
        data.to_csv('download/'+music_name+'.csv',encoding='utf_8_sig')
        return music_hot_comments

    

In [154]:
def start():
    search_data_paras = '{{"hlpretag":"<span class=\\"s-fc7\\">","hlposttag":"</span>"'\
                ',"s":"{}","type":"1",'\
                '"offset":"0","total":"true","limit":"30","csrf_token":""}}'
    music_down_sequence_paras = '{{"ids":"[{}]","level":"standard","encodeType":"aac","csrf_token":""}}'
    music_comments_sequence_paras = '{{"rid":"R_SO_4_{0}",'\
                            '"threadId":"R_SO_4_{1}","pageNo":"1","pageSize":"20"'\
                            ',"cursor":"-1","offset":"0","orderType":"1","csrf_token":""}}'

    encrpter = Encrypter()
    music_downloader = MusicDownloader()

    # s_ = input('a singer or song:')
    search_data = search_data_paras.format('张国荣')
    form_data = encrpter.get_encrypt(search_data)
    music_dic = music_downloader.get_dic(form_data)

    # print(music_dic)
    ## 使用pandas存储表格,便于后续使用
    song_ids = []
    song_names = []
    for songid,songname in music_dic.items():
        song_ids.append(songid)
        song_names.append(songname)
    pd_music = pd.DataFrame(
        {'song_id':song_ids,'song_names':song_names}
    )
    print('下面将下载以下歌曲.......')
    print(pd_music)
    for key,item in music_dic.items():
        
        song_id = key

        music_down_sequence=music_down_sequence_paras.format(song_id)
        form_data = encrpter.get_encrypt(music_down_sequence)
        music_downloader.download_music(form_data,music_dic[song_id])

        music_comments_sequence = music_comments_sequence_paras.format(song_id,song_id)
        form_data = encrpter.get_encrypt(music_comments_sequence)
        text = music_downloader.download_hot_comment(form_data,music_dic[song_id])
        if text:
            continue
            # print(text)
        else:
            print('没有热评')

In [155]:
start()

下面将下载以下歌曲.......
       song_id    song_names
0   1895800467     沉默是金 (合唱)
1       188204          沉默是金
2       188459    千千阙歌(Live)
3       186436          玻璃之情
4   1397657356    当年情 (Live)
5       188489    风继续吹(Live)
6     26620756        当爱已成往事
7       186453          春夏秋冬
8       188175          倩女幽魂
9       187076  Monica(Live)
10    29343374        怪你过份美丽
11      186760          风再起时
12    29343372           有心人
13      187374             我
14      186421             追
15      186707            想你
16      186699          共同渡过
17    26075352       月亮代表我的心
18      186689          无心睡眠
19     5244302          有谁共鸣
没有热评
         user                                            content  likedCount
0       汉堡包爱我  我爸最喜欢的歌，07年的时候负债400多万，很多人看不起他，他就默默地做事，就在前几天把欠的...      156395
1     大明浮生梦游记                            中国十几亿人   却再不会有一个哥哥了[哀伤]       50311
2     張國榮是靚仔呀      大家跟我一起做：网易云音乐-设置-意见反馈-输入“强烈要求补全张国荣的所有歌版权！”谢谢!       37746
3     葉來風雨聲-古  张国荣先生是一个100分的好人。曾经有一句话这么形容他:与他的才华相比,他的容貌不值一提,与

In [121]:
'鑳剁墖浜篲'.encode('GBK').decode()

'胶片人_'

In [142]:
# i = 'http://m10.music.126.net/20211119150419/0b0ba3b87216e93144d5f9bf79977fbb/yyaac/obj/wonDkMOGw6XDiTHCmMOi/3058352428/28ac/5055/a810/16dd4fbad431d482ad28049b9e1b4f5f.m4a'
# data = requests.get(i)
# with open('download/q.m4a','wb') as f:
#     f.write(data.content)

In [130]:
# print(temp)

[[1895800467, '沉默是金 (合唱)'], [188204, '沉默是金'], [188459, '千千阙歌(Live)'], [186436, '玻璃之情'], [1397657356, '当年情 (Live)'], [188489, '风继续吹(Live)'], [26620756, '当爱已成往事'], [186453, '春夏秋冬'], [188175, '倩女幽魂'], [187076, 'Monica(Live)'], [29343374, '怪你过份美丽'], [186760, '风再起时'], [29343372, '有心人'], [187374, '我'], [186707, '想你'], [26075352, '月亮代表我的心'], [186699, '共同渡过'], [186689, '无心睡眠'], [186421, '追'], [186626, '胭脂扣']]
